In [22]:
import os

spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [23]:
# Start Spark session
from pyspark.sql import SparkSession
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-09 20:07:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.25MB/s    in 0.7s    

2022-11-09 20:07:16 (1.25 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [24]:
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract


In [25]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://county-pop-df.s3.us-west-2.amazonaws.com/health_outcomes_df.csv"
spark.sparkContext.addFile(url)
health_outcomes_df = spark.read.csv(SparkFiles.get("health_outcomes_df.csv"), sep=",", header=True,)

In [26]:
url = "https://county-pop-df.s3.us-west-2.amazonaws.com/health_risk_behaviors_df.csv"
spark.sparkContext.addFile(url)
health_risk_behaviors_df = spark.read.csv(SparkFiles.get("health_risk_behaviors_df.csv"), sep=",", header=True,)

In [27]:
url = "https://county-pop-df.s3.us-west-2.amazonaws.com/health_status_df.csv"
spark.sparkContext.addFile(url)
health_status_df = spark.read.csv(SparkFiles.get("health_status_df.csv"), sep=",", header=True,)

In [28]:
url = "https://county-pop-df.s3.us-west-2.amazonaws.com/prevention_df.csv"
spark.sparkContext.addFile(url)
prevention_df = spark.read.csv(SparkFiles.get("prevention_df.csv"), sep=",", header=True,)

In [29]:
url = "https://county-pop-df.s3.us-west-2.amazonaws.com/population_df.csv"
spark.sparkContext.addFile(url)
population_df = spark.read.csv(SparkFiles.get("population_df.csv"), sep=",", header=True,)

In [30]:
url = "https://county-pop-df.s3.us-west-2.amazonaws.com/county_info_df.csv"
spark.sparkContext.addFile(url)
county_info_df = spark.read.csv(SparkFiles.get("county_info_df.csv"), sep=",", header=True,)

In [10]:
health_outcomes_df.show(5)

+----------+---------+-------+------+------+--------+------+----+---+----------+--------+-------+---------+------+
|CountyFIPS|ARTHRITIS|CASTHMA|BPHIGH|CANCER|HIGHCHOL|KIDNEY|COPD|CHD|DEPRESSION|DIABETES|OBESITY|TEETHLOST|STROKE|
+----------+---------+-------+------+------+--------+------+----+---+----------+--------+-------+---------+------+
|     40065|     23.0|    9.7|  34.1|   6.1|    31.3|   3.1| 7.1|6.3|      21.2|    11.8|   36.3|     18.8|   3.3|
|     17101|     25.6|    9.5|  33.3|   6.5|    29.3|   2.9| 8.0|6.4|      21.2|    10.7|   38.8|     17.2|   3.4|
|     20009|     23.7|    9.7|  32.9|   6.4|    30.4|   2.9| 6.7|5.8|      19.0|    11.0|   37.0|     17.2|   3.1|
|      2020|     20.0|    9.2|  29.8|   6.0|    27.8|   2.5| 5.0|4.8|      17.0|     8.3|   30.7|     12.2|   2.7|
|     40137|     25.7|   10.5|  36.0|   6.6|    30.0|   3.0| 8.4|6.8|      23.9|    10.5|   36.5|     20.6|   3.5|
+----------+---------+-------+------+------+--------+------+----+---+----------+

In [36]:
health_outcomes_df.printSchema()

root
 |-- CountyFIPS: string (nullable = true)
 |-- ARTHRITIS: string (nullable = true)
 |-- CASTHMA: string (nullable = true)
 |-- BPHIGH: string (nullable = true)
 |-- CANCER: string (nullable = true)
 |-- HIGHCHOL: string (nullable = true)
 |-- KIDNEY: string (nullable = true)
 |-- COPD: string (nullable = true)
 |-- CHD: string (nullable = true)
 |-- DEPRESSION: string (nullable = true)
 |-- DIABETES: string (nullable = true)
 |-- OBESITY: string (nullable = true)
 |-- TEETHLOST: string (nullable = true)
 |-- STROKE: string (nullable = true)



In [11]:
health_risk_behaviors_df.show(5)

+----------+-----+--------+----+-----+
|CountyFIPS|BINGE|CSMOKING| LPA|SLEEP|
+----------+-----+--------+----+-----+
|     40065| 13.6|    17.4|38.5| 37.7|
|     17101| 21.5|    21.7|30.3| 36.8|
|     20009| 17.1|    18.7|32.1| 34.2|
|      2020| 18.2|    15.3|20.5| 34.4|
|     40137| 13.5|    21.8|39.1| 37.3|
+----------+-----+--------+----+-----+
only showing top 5 rows



In [12]:
health_status_df.show(5)

+----------+-----+-----+-----+
|CountyFIPS|MHLTH|PHLTH|GHLTH|
+----------+-----+-----+-----+
|     40065| 15.0| 13.8| 22.6|
|     17101| 15.8| 13.9| 20.9|
|     20009| 15.4| 12.5| 20.0|
|      2020| 12.4| 10.1| 15.3|
|     40137| 16.7| 14.6| 22.2|
+----------+-----+-----+-----+
only showing top 5 rows



In [13]:
prevention_df.show(5)

+----------+------+-------+------+-----+----------+--------+--------+------------+-----+-----+
|CountyFIPS|ACCESS|CHECKUP|DENTAL|BPMED|CHOLSCREEN|MAMMOUSE|CERVICAL|COLON_SCREEN|COREM|COREW|
+----------+------+-------+------+-----+----------+--------+--------+------------+-----+-----+
|     40065|  24.2|   74.1|  61.1| 57.9|      83.8|    71.5|    83.0|        58.7| 25.2| 23.7|
|     17101|  18.9|   73.4|  57.8| 56.8|      83.0|    68.7|    82.2|        61.2| 24.7| 20.6|
|     20009|  20.5|   72.6|  59.3| 56.8|      82.1|    69.0|    82.6|        58.4| 24.4| 22.3|
|      2020|  13.6|   68.8|  68.5| 48.5|      81.2|    67.4|    84.1|        58.4| 20.8| 25.1|
|     40137|  20.5|   74.5|  58.3| 57.7|      82.8|    69.6|    82.3|        54.6| 25.5| 27.2|
+----------+------+-------+------+-----+----------+--------+--------+------------+-----+-----+
only showing top 5 rows



In [14]:
population_df.show(5)

+----------+-------+----------------+----------+
|CountyFIPS|Density|total_population|RuralUrban|
+----------+-------+----------------+----------+
|      1001|   94.3|           56059|     Rural|
|      1003|  141.8|          225463|     Rural|
|      1005|   27.8|           24575|     Rural|
|      1007|   36.0|           22427|     Rural|
|      1009|   89.9|           57997|     Rural|
+----------+-------+----------------+----------+
only showing top 5 rows



In [15]:
county_info_df.show(5)

+----------+----------+---------+
|CountyFIPS|CountyName|StateDesc|
+----------+----------+---------+
|     40065|   Jackson| Oklahoma|
|     17101|  Lawrence| Illinois|
|     20009|    Barton|   Kansas|
|      2020| Anchorage|   Alaska|
|     40137|  Stephens| Oklahoma|
+----------+----------+---------+
only showing top 5 rows



# Load

In [31]:
from getpass import getpass

password = getpass('enter pass words here')

enter pass words here··········


In [32]:
#configure settings for RDS
mode="append"
jdbc_url = "jdbc:postgresql://b-team-final-project.cct7ahzel1ur.us-west-2.rds.amazonaws.com:5432/B_Team_Final_Project"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [33]:
#load DataFrames into pgAdmin
# Write population_df to county_population table in pg
population_df.write.jdbc(url=jdbc_url, table='county_population', mode=mode, properties=config)

In [34]:
# Write county_info_df to county_info table 
county_info_df.write.jdbc(url=jdbc_url, table='county_info', mode=mode, properties=config)

In [37]:
# Write health_outcomes_df to health_outcomes table 
health_outcomes_df.write.jdbc(url=jdbc_url, table='health_outcomes', mode=mode, properties=config)

In [38]:
# Write health_risk_behaviors_df to health_risk_behaviors table 
health_risk_behaviors_df.write.jdbc(url=jdbc_url, table='health_risk_behaviors', mode=mode, properties=config)

In [39]:
# Write health_status_df to health_status table 
health_status_df.write.jdbc(url=jdbc_url, table='health_status', mode=mode, properties=config)

In [40]:
# Write prevention_df to prevention table 
prevention_df.write.jdbc(url=jdbc_url, table='prevention', mode=mode, properties=config)